In [30]:
# 必要なモジュールの import
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import requests
from dotenv import load_dotenv
import os

In [31]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")


chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE,
    model_name="gpt-3.5-turbo",
    temperature=0,
)

In [32]:
def get_commit(commit_sha):
    access_token = os.getenv("GITHUB_ACCESS_TOKEN")
    repo_owner = "imoken777"
    repo_name = "Gradius-2023"

    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/commits/{commit_sha}"
    headers = {"Authorization": f"token {access_token}"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        response_data = response.json()
        files = response_data.get("files", [])

        commit_info = ""

        for file in files:
            patch = file.get("patch")
            if patch:
                commit_info += f"File: {file['filename']}\nPatch:\n{patch}\n\n"
            else:
                commit_info += f"File: {file['filename']} - No patch available\n\n"

        return commit_info
    else:
        return f"Error: {response.status_code}, {response.text}"


In [33]:
tools = [
    Tool.from_function(
        func=get_commit,
        name="get_commit",
        description="指定されたコミットの詳細情報を取得するための関数です。",
    )
]

agent = initialize_agent(
    tools,
    chat,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)


In [34]:
agent.run(
    "以下のコミットの内容を日本語で詳しく説明してください。141b2193d2ac846077460f04fa68190772b201eb"
)

'このコミットは、controller.ts、index.ts、bulletUsecase.tsのファイルに変更が加えられています。具体的には、controller.tsのgetメソッドとpostメソッドが更新され、index.tsのpostメソッドのresBodyの型が変更され、bulletUsecase.tsのcreateメソッドの引数と処理が変更されました。'